In [1]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import cmath
from scipy.optimize import curve_fit
from scipy.stats import variation
from statistics import mean, median

Paste your 96 well format OD data from Excel between the upper ''' and lower '''.

In [2]:
rawData = '''
99611940.2	87957537.86	95879637.03	624133.0935	72440724.42	107369.0882	92171.88112	100239.2555	94510.24468	138828.006	129090.4065	53383.81503
55236673.82	50031394.99	56547174.79	531842.0613	60852.94465	87240.69545	84196.33992	103651.575	102330.3552	151014.0775	146314.6818	58660.54111
8473963.492	7717064.539	8464353.125	577045.4927	58339.96861	90013.71086	87886.80569	100421.8198	100296.9073	144028.1363	149239.3152	58918.68065
613508.8136	637392.4919	614753.8525	19360852.56	55105.47798	90043.73521	82087.55531	99643.52117	95747.3026	140970.2679	139110.3545	54670.85772
136876.816	165840.4766	160171.9	18822795.36	58219.90424	92545.39158	88239.88605	97700.19967	99736.00392	138284.9375	149569.753	51683.78812
108923.3925	128241.0336	129224.9612	18462274.6	63513.66119	91214.691	89695.45322	102548.9559	99010.55687	144998.9814	143436.9822	55881.07512
92993.36554	111335.3683	112881.3176	70084232.55	69195.50492	97606.5175	86774.72972	106382.9446	95858.99878	146858.9897	134095.4661	52813.53485
16133.031	31787.17096	32066.85708	71590408.17	53716.37972	90244.29808	89633.00276	105370.3844	120805.984	154754.7458	150779.7641	58476.84215
'''
row_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
column_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

plateTemplate = pd.DataFrame(index = row_list, columns = column_list)

rawData = pd.read_table(io.StringIO(rawData), delim_whitespace=True, names=plateTemplate.columns)
rawData.index = plateTemplate.index
rawData

,1,2,3,4,5,6,7,8,9,10,11,12
A,9.961194e+07,8.795754e+07,9.587964e+07,6.241331e+05,7.244072e+07,107369.08820,92171.88112,100239.25550,94510.24468,138828.0060,129090.4065,53383.81503
B,5.523667e+07,5.003139e+07,5.654717e+07,5.318421e+05,6.085294e+04,87240.69545,84196.33992,103651.57500,102330.35520,151014.0775,146314.6818,58660.54111
C,8.473963e+06,7.717065e+06,8.464353e+06,5.770455e+05,5.833997e+04,90013.71086,87886.80569,100421.81980,100296.90730,144028.1363,149239.3152,58918.68065
D,6.135088e+05,6.373925e+05,6.147539e+05,1.936085e+07,5.510548e+04,90043.73521,82087.55531,99643.52117,95747.30260,140970.2679,139110.3545,54670.85772
E,1.368768e+05,1.658405e+05,1.601719e+05,1.882280e+07,5.821990e+04,92545.39158,88239.88605,97700.19967,99736.00392,138284.9375,149569.7530,51683.78812
F,1.089234e+05,1.282410e+05,1.292250e+05,1.846227e+07,6.351366e+04,91214.69100,89695.45322,102548.95590,99010.55687,144998.9814,143436.9822,55881.07512
G,9.299337e+04,1.113354e+05,1.128813e+05,7.008423e+07,6.919550e+04,97606.51750,86774.72972,106382.94460,95858.99878,146858.9897,134095.4661,52813.53485
H,1.613303e+04,3.178717e+04,3.206686e+04,7.159041e+07,5.371638e+04,90244.29808,89633.00276,105370.38440,120805.98400,154754.7458,150779.7641,58476.84215


Paste your 96 well format plate template between the upper ''' and lower '''.

In [3]:
plateLayout = '''
A	A	1	1	9	9	17	17	25	25	33	33
B	B	2	2	10	10	18	18	26	26	34	34
C	C	3	3	11	11	19	19	27	27	35	35
D	D	4	4	12	12	20	20	28	28	36	36
E	E	5	5	13	13	21	21	29	29	37	37
F	F	6	6	14	14	22	22	30	30	38	38
G	G	7	7	15	15	23	23	31	31	39	39
H	H	8	8	16	16	24	24	32	32	40	40
'''

plateLayout = pd.read_table(io.StringIO(plateLayout), delim_whitespace=True, names=plateTemplate.columns)
plateLayout.index = plateTemplate.index
plateLayout

,1,2,3,4,5,6,7,8,9,10,11,12
A,A,A,1,1,9,9,17,17,25,25,33,33
B,B,B,2,2,10,10,18,18,26,26,34,34
C,C,C,3,3,11,11,19,19,27,27,35,35
D,D,D,4,4,12,12,20,20,28,28,36,36
E,E,E,5,5,13,13,21,21,29,29,37,37
F,F,F,6,6,14,14,22,22,30,30,38,38
G,G,G,7,7,15,15,23,23,31,31,39,39
H,H,H,8,8,16,16,24,24,32,32,40,40


Check that the standards values listed below are correct.

In [4]:
# In the [] after pd.Series, input the standard concentrations. Number separated by a comma.
# In the [] after index=, input the corresponding labels for each standard. Labels in 'quotes' separated by a comma.
standardConc = pd.Series([9661, 4026, 685.4, 137.5, 35.55, 17.97, 8.64, 4.32], index=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], name='Conc')

In [5]:
wellID = []
sampleID = []
wellOD = []
for row_data in row_list:
    for column_data in column_list:
        wellID.append(row_data + column_data)
        sampleID.append(plateLayout.at[row_data, column_data])
        wellOD.append(rawData.at[row_data, column_data])

dataTable = pd.DataFrame({'Sample ID':sampleID, 'OD':wellOD}, index=wellID)

standardCurve = dataTable.groupby('Sample ID').mean()
standardCurve = pd.concat([standardConc, standardCurve], axis=1, join_axes=[standardConc.index])
standardCurve = standardCurve.values

In [6]:
type(standardCurve)

numpy.ndarray

In [7]:
standardCurve.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False